In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [7]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = torch.logical_and(x>20, x<60)
    filtered_x = s.where(filter, x)
    return s.harmonic_mean(filtered_x)

error = 0.1

_, prover_model = computation_to_model(computation, precal_witness_path, True, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(True)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is depre

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 7, param_scale: 7, scale_input_multiplier: 1) ------------->

+---------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error  | max_error | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.0010948181 | -0.0021896362 | 0         | -0.0021896362 | 0.0010948181   | 0.0021896362     | 0.0021896362  | 0             | 0.0000023972534    | -0.00002650731     | 0.00002650731          |
+---------------+---------------+-----------+---------------+----------------+--------------

scale:  [7]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":1,"lookup_range":[-7168,17408],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":15331,"total_const_size":5418,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[[-64,64],[64,192]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715153384764}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False,error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 3.7415249347686768 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 41.3046875
==== Generating Proof ====
proof:  {'instances': [['b3fbe4e7fbfe1e847f816b6156995c7d4bb640b4687064937bfafe65f0167d0f', '0100000000000000000000000000000000000000000000000000000000000000', 'a714000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x05cd3b7e04d922db43eb3b8bd6f24809526f37d810770094d1671fde6cb7f32f28ff9abf2e6357e5df6412ff6880caace0a003506f7f489194395a8fce120f71103a6aea8da905c1c71d8d1ca525ceafa5efd47c3bd6aeb0d3ff126260f9a52f25b292e4f0fbb208727aa9ff50950c6a6ad62e3a1d82e80f4f5665f578c2b87510d0bc8c5e2896ae78b7e86e0ce3b5ad56c0085059c3f9c8865651700332b5491d8bc1859deee49efce0c5ee5bd22bc6ecc96c782295ca9161dfa8a1d13ec5aa0be3d6e33d1fdde1d574a1379b723da1d2db9a62df20bd503bd5df3054fcb0821cefd079a776bcb6c03c6cccbd8218ec70edb5a2c41a4732a96d191e8330ea7321ec9179da9152f6f9c0a7cbae9a077183d3d7224f770e6c352e42856a972a3022a65f714

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [41.3046875]
